## If person ever worked in a particular org, extract info for that person

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../../')

In [3]:
import os
import pandas as pd

from tqdm import tqdm

from utils.db_util import create_postgres_engine
from utils.query_util import query_person_info_in_organization

In [4]:
db_conn = create_postgres_engine()

In [5]:
company_df = pd.read_csv('../../data/query_result/filtered_org_stat_50_word_count_cutoff.csv')

In [6]:
company_df.shape

(1428, 4)

In [7]:
def escape_single_quote(text):
    """Since postgres does not like single quote in text"""
    text = text.replace("'", "''")
    return text

In [8]:
OUTPUT_FILE_DIR = '/home/sjb/Projects/Research/LinkedIn_OB/data/company_level_individual_profiles/'
OUTPUT_FILE_NAME_FORMAT = '{short_name}_person_profile_50_word_cutoff.csv'

ORG_PROFILE_SUFFIX = '\?trk=ppro_cprof'
ORG_PROFILE_PREFIX = '/company/'

In [9]:
company_df['short_company_name'] = company_df['org_profile_link'].str.replace(ORG_PROFILE_PREFIX, '').str.replace(ORG_PROFILE_SUFFIX, '')

In [10]:
for _, row in tqdm(company_df.iterrows()):
    short_company_name = row['short_company_name']
    count_person = row['count_person']
    
    org_profile_link = row['org_profile_link']
    org_profile_link_escaped = escape_single_quote(org_profile_link)
    
    query_result = query_person_info_in_organization(db_conn, org_profile_link_escaped)
    
    # check sum
    assert query_result.shape[0] == count_person
    
    # save result
    output_file_path = os.path.join(OUTPUT_FILE_DIR,
                                    OUTPUT_FILE_NAME_FORMAT.format(short_name=short_company_name))
    query_result.to_csv(output_file_path, index=False)

1428it [2:17:05,  5.76s/it]


In [11]:
query_result.shape

(1001, 13)

In [ ]:
# temp_path = '/home/sjb/Projects/Research/LinkedIn_OB/data/company_level_individual_profiles/us-army_person_profile_50_word_cutoff.csv'